# Basics

In [1]:
words = open(r"..\data\names.txt", 'r').read().splitlines()

# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [118]:
import torch
import torch.nn.functional as F

block_size = 3

def build_dataset(words):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X,Y

import random
random.seed(42)
random.shuffle(words)

n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

X, Y = build_dataset(words)

torch.Size([182400, 3]) torch.Size([182400])
torch.Size([22962, 3]) torch.Size([22962])
torch.Size([22784, 3]) torch.Size([22784])
torch.Size([228146, 3]) torch.Size([228146])


In [127]:
g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 10), generator=g)
W1 = torch.randn((30, 200), generator=g)
b1 = torch.randn(200, generator=g)
W2 = torch.randn((200, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

for p in parameters:
    p.requires_grad = True

for _ in range(1):

    ix = torch.randint(0, X.shape[0], (32,))
    print(ix)
    emb = C[X[ix]]
    print(emb.shape)
    h = torch.tanh(emb.view(-1,30) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits,Y)
    print(loss.item())
    for p in parameters:
        p.grad = None 
    loss.backward()
    for p in parameters:
        p.data += -0.1 * p.grad


tensor([148618, 188729, 117446,  17697, 167884,  68866, 182530,  85179,  66568,
        165420,  60581, 167999, 165187, 146988,  36197, 148882,  50102,  45900,
         68467,  38489, 126354,  67734,  77032, 162412,   4944, 158376,  72552,
         85488, 215929,  19871,  24768,  13716])
torch.Size([32, 3, 10])


ValueError: Expected input batch_size (32) to match target batch_size (228146).

In [117]:
logits.max(1)

torch.return_types.max(
values=tensor([10.8022, 25.0367, 10.8022, 20.6390, 10.8022, 27.9046, 10.8022, 21.3068,
        10.8022, 18.4426, 10.8022, 25.0367, 10.8022, 23.1714],
       grad_fn=<MaxBackward0>),
indices=tensor([1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]))

In [87]:
Y[0]

tensor(1)